In [4]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import numpy as np
from IPython.display import display, HTML
from google.colab import files # Colab에서 파일 다운로드를 위해 필요

# 1. 모델 및 토크나이저 로드 (skt/kogpt2-base-v2 사용)
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME, output_attentions=True, attn_implementation="eager")

print(f"로드된 모델: {MODEL_NAME}")
print(f"모델 레이어 수: {model.config.n_layer}")
print(f"모델 헤드 수: {model.config.n_head}")

# --- 어텐션 가중치를 표로 출력하고 분석하는 함수 (HTML 문자열 반환으로 변경) ---
def get_attention_table_and_analyze_html(tokens, attention_weights, layer_idx, head_idx, sentence_context=""):
    """
    어텐션 가중치를 HTML 테이블로 표시하고, 해당 헤드의 잠재적 의미를 분석하여 HTML 문자열로 반환합니다.
    Args:
        tokens (list): 토큰 문자열 리스트.
        attention_weights (torch.Tensor): 어텐션 가중치 텐서 (seq_len, seq_len).
        layer_idx (int): 현재 레이어의 인덱스 (0-based).
        head_idx (int): 현재 헤드의 인덱스 (0-based).
        sentence_context (str): 문장 전체에 대한 맥락 제공.
    Returns:
        str: 생성된 HTML 문자열.
    """
    html_output = "" # 이 함수에서 생성된 HTML을 누적할 변수

    title = f"<h3>레이어 {layer_idx + 1}, 헤드 {head_idx + 1} 어텐션 가중치</h3>"

    # 'Ġ' 제거하여 가독성 높임
    header_tokens = [token.replace('Ġ', '') for token in tokens]
    header = "<th>Query / Key</th>" + "".join([f"<th>{t}</th>" for t in header_tokens])

    rows = []
    for i, query_token in enumerate(tokens):
        row_data = "".join([f"<td>{val:.3f}</td>" for val in attention_weights[i].tolist()])
        rows.append(f"<tr><th>{query_token.replace('Ġ', '')}</th>{row_data}</tr>")

    table_html = f"""
    {title}
    <table style="width:100%; border-collapse: collapse; text-align: center;">
        <tr style="background-color:#f2f2f2;">{header}</tr>
        {''.join(rows)}
    </table>
    """
    html_output += table_html # HTML 내용 누적

    # --- 어텐션 스코어 분석 ---
    analysis_html = f"<h4>레이어 {layer_idx + 1}, 헤드 {head_idx + 1} 분석:</h4>"
    analysis_points = []

    # 가장 높은 스코어 추출
    max_scores, max_indices = torch.max(attention_weights, dim=1)

    for i, query_token in enumerate(tokens):
        # 특수 토큰 제외 및 'Ġ' 제거
        cleaned_query_token = query_token.replace('Ġ', '')
        if cleaned_query_token.strip() in ['<s>', '</s>', '.', ',', '!', '?', '<pad>']:
            continue

        target_token = tokens[max_indices[i]].replace('Ġ', '')
        score = max_scores[i].item()

        # 스코어 임계값에 따른 분석 (조정 가능)
        if score > 0.7: # 매우 높은 스코어
            if cleaned_query_token == target_token:
                analysis_points.append(f"<b>'{cleaned_query_token}'</b>은(는) 자기 자신에게 매우 높은 주의({score:.3f})를 기울입니다. 이는 토큰의 고유한 의미나 위치 정보를 강력하게 강화하는 패턴일 수 있습니다.")
            else:
                analysis_points.append(f"<b>'{cleaned_query_token}'</b>은(는) <b>'{target_token}'</b>에 매우 높은 주의({score:.3f})를 기울입니다. 이는 두 토큰 간의 강력한 문법적/의미적 연결 (예: 주어-동사, 동사-목적어, 수식-피수식어)을 포착하는 헤드일 가능성이 큽니다.")
        elif score > 0.4: # 중간 스코어
             if cleaned_query_token == target_token:
                 analysis_points.append(f"<b>'{cleaned_query_token}'</b>은(는) 자기 자신에게 중간 수준의 주의({score:.3f})를 기울이며, 주변 토큰에도 분산됩니다. 이는 자신의 의미를 기반으로 주변 맥락을 넓게 탐색하는 패턴일 수 있습니다.")
             else:
                 analysis_points.append(f"<b>'{cleaned_query_token}'</b>은(는) <b>'{target_token}'</b>에 중간 수준의 주의({score:.3f})를 기울입니다. 이 헤드는 특정 핵심 관계와 더불어 광범위한 문맥 정보를 수집하거나, 여러 관련어 중 주요 후보를 탐색하는 데 기여할 수 있습니다.")
        else: # 낮은 스코어 (고르게 분포된 경우)
             analysis_points.append(f"<b>'{cleaned_query_token}'</b>은(는) 다양한 토큰에 비교적 고르게 주의를 분산({score:.3f} 이하)하고 있습니다. 이 헤드는 특정 핵심 관계보다는 문장 전체의 전반적인 맥락이나 흐름을 파악하는 데 초점을 맞출 수 있습니다.")

    if not analysis_points:
        analysis_html += "<p>이 헤드는 주로 특수 토큰에 집중하거나, 모든 토큰에 매우 고르게 주의를 분산하여 명확한 문법적/맥락적 패턴이 두드러지지 않습니다.</p>"
    else:
        analysis_html += "<ul>" + "".join([f"<li>{point}</li>" for point in analysis_points]) + "</ul>"

    analysis_html += "<p><i>(참고: 학습된 모델의 어텐션 패턴 분석입니다. 각 헤드는 문장의 다른 측면을 포착하며, 낮은 레이어는 문법적 관계에, 높은 레이어는 더 추상적인 의미에 집중하는 경향이 있습니다.)</i></p><hr>"
    html_output += analysis_html # HTML 내용 누적

    return html_output # 생성된 HTML 반환


# --- 어텐션 시각화 및 분석 함수 (HTML 파일 저장) ---
def analyze_attention_and_save_html(sentence, file_name="attention_analysis.html"):
    print(f"\n--- 문장: '{sentence}' ---")

    # 토큰화 및 모델 입력 준비
    inputs = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # 모델 실행하여 어텐션 가중치 가져오기
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        attentions = outputs.attentions

    # 토큰 문자열로 변환 (시각화를 위해)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    print(f"토큰화된 문장: {tokens}")
    print(f"총 {len(attentions)}개 레이어의 어텐션 정보.")

    # HTML 파일에 쓸 내용을 저장할 리스트
    all_html_content = []

    # 전체 파일의 헤더 및 스타일
    all_html_content.append(f"""
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>어텐션 분석 결과 - '{sentence}'</title>
        <style>
            body {{ font-family: 'Malgun Gothic', 'Apple Color Emoji', 'Segoe UI Emoji', sans-serif; line-height: 1.6; margin: 20px; background-color: #f4f4f4; color: #333; }}
            h1 {{ color: #0056b3; }}
            h3 {{ color: #007bff; margin-top: 30px; border-bottom: 1px solid #eee; padding-bottom: 5px; }}
            h4 {{ color: #28a745; margin-top: 15px; }}
            table {{ width: 100%; border-collapse: collapse; margin-top: 10px; background-color: #fff; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: center; }}
            th {{ background-color: #f8f8f8; }}
            .analysis-section {{ margin-bottom: 20px; padding: 10px; background-color: #e9ecef; border-left: 5px solid #007bff; }}
            ul {{ list-style-type: disc; margin-left: 20px; }}
            li {{ margin-bottom: 5px; }}
            hr {{ border: none; border-top: 1px dashed #ccc; margin: 40px 0; }}
        </style>
    </head>
    <body>
        <h1>문장: '{sentence}' 어텐션 분석 결과</h1>
        <p><b>토큰화된 문장:</b> {tokens}</p>
        <p><b>총 레이어 수:</b> {len(attentions)}개</p>
        <p><b>어텐션 스코어 해석:</b> 각 표는 특정 레이어와 헤드에서의 어텐션 가중치를 보여줍니다. 왼쪽 'Query' 토큰이 상단 'Key' 토큰에 얼마나 주의를 기울이는지 스코어로 나타납니다. 스코어가 높을수록 해당 Key 토큰의 정보가 Query 토큰의 표현에 더 중요하게 사용됩니다.</p>
        <hr>
    """)

    # 모든 레이어 (0부터 model.config.n_layer - 1)
    for layer_idx, layer_attentions in enumerate(attentions):
        # layer_attentions는 (batch_size, num_heads, seq_len, seq_len)
        # 배치 사이즈가 1이므로 squeeze(0)하여 (num_heads, seq_len, seq_len)로 만듭니다.
        heads_attentions = layer_attentions.squeeze(0)

        # 각 헤드 (0부터 model.config.n_head - 1)
        for head_idx, attention_matrix in enumerate(heads_attentions):
            # attention_matrix는 (seq_len, seq_len)
            html_chunk = get_attention_table_and_analyze_html(
                tokens, attention_matrix, layer_idx, head_idx, sentence
            )
            all_html_content.append(html_chunk) # 생성된 HTML 청크를 리스트에 추가

    all_html_content.append("</body></html>") # HTML 푸터 추가

    # 모든 HTML 내용을 하나의 문자열로 합치기
    final_html_output = "\n".join(all_html_content)

    # HTML 파일로 저장
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(final_html_output)
    print(f"\n분석 결과가 '{file_name}' 파일로 저장되었습니다.")

    # Colab에서 파일 다운로드 링크 제공
    files.download(file_name)


# 예제 문장 1
analyze_attention_and_save_html("우리팀은 지난 날의 연패를 끊고 이제는 연패의 길을 걷는다.", "school_attention_analysis.html")

# 다른 예제 문장 추가 가능
# analyze_attention_and_save_html("철수가 밥을 먹는다.", "korean_sentence_1_attention.html")
# analyze_attention_and_save_html("파란 하늘에 떠 있는 하얀 구름이 예쁘다.", "korean_sentence_2_attention.html")

로드된 모델: skt/kogpt2-base-v2
모델 레이어 수: 12
모델 헤드 수: 12

--- 문장: '우리팀은 지난 날의 연패를 끊고 이제는 연패의 길을 걷는다.' ---
토큰화된 문장: ['▁우리', '팀은', '▁지난', '▁날의', '▁연', '패를', '▁끊고', '▁이제는', '▁연', '패의', '▁길을', '▁걷는', '다.']
총 12개 레이어의 어텐션 정보.

분석 결과가 'school_attention_analysis.html' 파일로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ASR모델 평가

모델이 예측한 문장과 실제 입력 문장을 비교하여 얼마나 모델이 정확한지에 대한 코드

In [5]:
# -------------------------------
# 편집 거리 + 연산 추적 함수
# -------------------------------

def levenshtein_ops(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0]*(n+1) for _ in range(m+1)]
    op = [[None]*(n+1) for _ in range(m+1)]  # 연산 기록

    for i in range(m+1):
        dp[i][0] = i
        op[i][0] = 'D' if i > 0 else None
    for j in range(n+1):
        dp[0][j] = j
        op[0][j] = 'I' if j > 0 else None

    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
                op[i][j] = 'E'  # 일치
            else:
                del_cost = dp[i-1][j] + 1
                ins_cost = dp[i][j-1] + 1
                sub_cost = dp[i-1][j-1] + 1
                min_cost = min(del_cost, ins_cost, sub_cost)
                dp[i][j] = min_cost
                if min_cost == sub_cost:
                    op[i][j] = 'S'  # 치환
                elif min_cost == del_cost:
                    op[i][j] = 'D'  # 삭제
                else:
                    op[i][j] = 'I'  # 삽입

    # 연산 추적
    i, j = m, n
    subs, ins, dels = 0, 0, 0
    while i > 0 or j > 0:
        if op[i][j] == 'E':
            i -= 1
            j -= 1
        elif op[i][j] == 'S':
            subs += 1
            i -= 1
            j -= 1
        elif op[i][j] == 'D':
            dels += 1
            i -= 1
        elif op[i][j] == 'I':
            ins += 1
            j -= 1

    return dp[m][n], subs, ins, dels

# -------------------------------
# 평가 함수 (WER, CER + 정확도 + 세부연산)
# -------------------------------

def wer_detail(reference, hypothesis):
    ref_words = reference.strip().split()
    hyp_words = hypothesis.strip().split()
    distance, subs, ins, dels = levenshtein_ops(ref_words, hyp_words)
    wer_score = distance / max(len(ref_words), 1)
    accuracy = (len(ref_words) - distance) / max(len(ref_words), 1)
    return wer_score, accuracy, subs, ins, dels

def cer_detail(reference, hypothesis):
    ref_chars = list(reference.strip().replace(" ", ""))
    hyp_chars = list(hypothesis.strip().replace(" ", ""))
    distance, subs, ins, dels = levenshtein_ops(ref_chars, hyp_chars)
    cer_score = distance / max(len(ref_chars), 1)
    accuracy = (len(ref_chars) - distance) / max(len(ref_chars), 1)
    return cer_score, accuracy, subs, ins, dels

# -------------------------------
# 예제 입력 및 출력
# -------------------------------

ref_text = "아버지가 방에 들어가시다"
hyp_text = "아버지 가방에 들어가시다"

print("Reference :", ref_text)
print("Hypothesis:", hyp_text)

# WER
wer_score, wer_acc, wer_s, wer_i, wer_d = wer_detail(ref_text, hyp_text)
print("\n[WER 평가 - 단어 단위]")
print("WER (오류율)      :", round(wer_score, 3))
print("정확도 (Accuracy) :", round(wer_acc, 3))
print("치환(Sub)         :", wer_s)
print("삽입(Ins)         :", wer_i)
print("삭제(Del)         :", wer_d)

# CER
cer_score, cer_acc, cer_s, cer_i, cer_d = cer_detail(ref_text, hyp_text)
print("\n[CER 평가 - 문자 단위]")
print("CER (오류율)      :", round(cer_score, 3))
print("정확도 (Accuracy) :", round(cer_acc, 3))
print("치환(Sub)         :", cer_s)
print("삽입(Ins)         :", cer_i)
print("삭제(Del)         :", cer_d)

Reference : 아버지가 방에 들어가시다
Hypothesis: 아버지 가방에 들어가시다

[WER 평가 - 단어 단위]
WER (오류율)      : 0.667
정확도 (Accuracy) : 0.333
치환(Sub)         : 2
삽입(Ins)         : 0
삭제(Del)         : 0

[CER 평가 - 문자 단위]
CER (오류율)      : 0.0
정확도 (Accuracy) : 1.0
치환(Sub)         : 0
삽입(Ins)         : 0
삭제(Del)         : 0


pre-trained ASR model 평가

In [1]:
# 1. 데이터 다운로드
!wget -O KoreanReadSpeechCorpus.tar.gz https://www.openslr.org/resources/97/KoreanReadSpeechCorpus.tar.gz

# 2. 압축 해제
!tar -xvzf KoreanReadSpeechCorpus.tar.gz

--2025-07-18 12:15:59--  https://www.openslr.org/resources/97/KoreanReadSpeechCorpus.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/97/KoreanReadSpeechCorpus.tar.gz [following]
--2025-07-18 12:16:00--  https://openslr.elda.org/resources/97/KoreanReadSpeechCorpus.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281297310 (268M) [application/x-gzip]
Saving to: ‘KoreanReadSpeechCorpus.tar.gz’

KoreanReadSpeechCor 100%[===================>] 268.27M  19.0MB/s    in 13s     

2025-07-18 12:16:13 (20.9 MB/s) - ‘KoreanReadSpeechCorpus.tar.gz’ saved [281297310/281297310]

AirbnbStudio/
AirbnbStudio/sub100100a00000.wav
Air

In [3]:
#전사 목록과 음원 목록 매핑
import json
import pandas as pd

# JSON 파일 로드
json_path = "/content/Korean_Read_Speech_Corpus_sample.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 모든 발화의 (파일 경로, 텍스트) 추출
entries = []
for location, utterances in data.items():
    for uid, info in utterances.items():
        wav_path = f"/content/{location}/{uid}.wav"
        text = info["text"]
        entries.append({"wav_path": wav_path, "text": text})

# DataFrame 생성
df = pd.DataFrame(entries)

# 결과 확인
print(df.head())

                                    wav_path  \
0  /content/AirbnbStudio/sub100100a00000.wav   
1  /content/AirbnbStudio/sub100100a00001.wav   
2  /content/AirbnbStudio/sub100100a00002.wav   
3  /content/AirbnbStudio/sub100100a00003.wav   
4  /content/AirbnbStudio/sub100100a00004.wav   

                                                text  
0                                저 식당 음식이 정말 맛있나 봐요.  
1                  아, 저기요. 삼계탕만 파는 식당인데 항상 사람들이 많아요.  
2   우리 회사 근처에 저런 유명한 식당이 있었네요. 다음에 삼계탕 한번 먹으러 가야겠어요.  
3  저 식당은 그날 준비한 걸 다 팔면 문을 닫아요. 그러니까 늦게 가면 못 드실 수도...  
4             여권을 만들어야 하는데요. 회사 일이 늦게 끝나서 갈 시간이 없어요.  


In [4]:
#전사된 문장과 음원 재생
from IPython.display import Audio, display

def play(index):
    if index < 0 or index >= len(df):
        print("잘못된 인덱스입니다.")
        return
    print(f"[{index}] 전사 문장:", df.iloc[index]["text"])
    display(Audio(df.iloc[index]["wav_path"], autoplay=False))

# 예시 실행
play(230)

[230] 전사 문장: 아니요. 월급 대부분을 생활비로 쓰느라 저축은 생각도 못 해요.


In [6]:
# 1. 필수 라이브러리 설치
!pip install -q transformers torchaudio librosa pandas

# 2. 라이브러리 임포트
import os, json
import pandas as pd
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from IPython.display import display

# 3. 평가 함수 정의
def levenshtein_ops(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    op = [[None] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
        op[i][0] = 'D' if i > 0 else None
    for j in range(n + 1):
        dp[0][j] = j
        op[0][j] = 'I' if j > 0 else None
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq1[i - 1] == seq2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
                op[i][j] = 'E'
            else:
                del_cost = dp[i - 1][j] + 1
                ins_cost = dp[i][j - 1] + 1
                sub_cost = dp[i - 1][j - 1] + 1
                dp[i][j] = min(del_cost, ins_cost, sub_cost)
                if dp[i][j] == sub_cost:
                    op[i][j] = 'S'
                elif dp[i][j] == del_cost:
                    op[i][j] = 'D'
                else:
                    op[i][j] = 'I'
    i, j = m, n
    subs, ins, dels = 0, 0, 0
    while i > 0 or j > 0:
        if op[i][j] == 'E': i -= 1; j -= 1
        elif op[i][j] == 'S': subs += 1; i -= 1; j -= 1
        elif op[i][j] == 'D': dels += 1; i -= 1
        elif op[i][j] == 'I': ins += 1; j -= 1
    return dp[m][n], subs, ins, dels

def wer_detail(ref, hyp):
    r, h = ref.strip().split(), hyp.strip().split()
    d, s, i, d_ = levenshtein_ops(r, h)
    return d / max(len(r),1), (len(r)-d)/max(len(r),1), s, i, d_

def cer_detail(ref, hyp):
    r, h = list(ref.replace(" ", "")), list(hyp.replace(" ", ""))
    d, s, i, d_ = levenshtein_ops(r, h)
    return d / max(len(r),1), (len(r)-d)/max(len(r),1), s, i, d_

# 4. 모델 목록 정의
model_list = {
    "kresnik-korean": "kresnik/wav2vec2-large-xlsr-korean",
    "kresnik-korean-v2": "kresnik/wav2vec2-large-xlsr-53-korean-v2",
    "xlsr-53": "facebook/wav2vec2-large-xlsr-53",
    "xlsr-300m": "facebook/wav2vec2-xls-r-300m",
    "xlsr-1b": "facebook/wav2vec2-xls-r-1b",
    "xlsr-2b": "facebook/wav2vec2-xls-r-2b",
    "960h": "facebook/wav2vec2-large-960h",
    "hubert-large-ls960-ft": "facebook/hubert-large-ls960-ft",
    "hubert-base-ls960": "facebook/hubert-base-ls960",
    "speechT5": "microsoft/speecht5_asr",
    "kocohub-wav2vec2-ko": "kocohub/wav2vec2-large-ko-ft",
    "etri-korean": "etri/wav2vec2-large-korean"
}

# 5. 미리 다운로드 (Processor + Model)
print("모델 다운로드 중...")
for key, path in model_list.items():
    try:
        _ = Wav2Vec2Processor.from_pretrained(path)
        _ = Wav2Vec2ForCTC.from_pretrained(path)
        print(f"{key} 다운로드 완료")
    except Exception as e:
        print(f"{key} 다운로드 실패:", e)

print("\n다운로드 완료. 이제 인식 평가 시작...\n")

# 6. JSON 데이터 로딩
json_path = "/content/Korean_Read_Speech_Corpus_sample.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

entries = []
for location, utterances in data.items():
    for uid, info in utterances.items():
        wav_path = f"/content/{location}/{uid}.wav"
        entries.append({"wav_path": wav_path, "text": info["text"]})
df = pd.DataFrame(entries)



모델 다운로드 중...
kresnik-korean 다운로드 완료
kresnik-korean-v2 다운로드 실패: kresnik/wav2vec2-large-xlsr-53-korean-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


xlsr-53 다운로드 실패: expected str, bytes or os.PathLike object, not NoneType
xlsr-300m 다운로드 실패: expected str, bytes or os.PathLike object, not NoneType
xlsr-1b 다운로드 실패: expected str, bytes or os.PathLike object, not NoneType
xlsr-2b 다운로드 실패: expected str, bytes or os.PathLike object, not NoneType


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


960h 다운로드 완료


...

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.


hubert-large-ls960-ft 평가 실패: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?
모델 평가: hubert-base-ls960
hubert-base-ls960 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: speechT5


You are using a model of type speecht5 to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at microsoft/speecht5_asr and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.layer_norm.bias', 'wav2vec2.encoder.layer_norm.weight', 'wav2vec2.encoder.layers.0.attention.k_proj.bias', 'wav2vec2.encoder.layers.0.attention.k_proj.weight', 'wav2vec2.encoder.layers.0.attention.out_proj.bias', 'wav2vec2.encoder.layers.0.attention.out_proj.weight', 'wav2vec2.encoder.layers.0.attention.q_proj.bias', 'wav2vec2.encoder.layers.0.attention.q_proj.weight', 'wav2vec2.encoder.layers.0.attention.v_proj.bias', 'wav2vec2.encoder.layers.0.attention.v_proj.weight', 'wav2vec2.encoder.layers.0.feed_forward.intermediate_dense.bias', 'wav2vec2.encoder.layers.0.feed_forward.intermediate_dense.weight', 'wav2vec2.encoder.layers.0.feed_forward.output_de

모델 평가: kocohub-wav2vec2-ko
kocohub-wav2vec2-ko 평가 실패: kocohub/wav2vec2-large-ko-ft is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
모델 평가: etri-korean
etri-korean 평가 실패: etri/wav2vec2-large-korean is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


,model,index,ref_text,pred_text,wer,wer_acc,wer_sub,wer_ins,wer_del,cer,cer_acc,cer_sub,cer_ins,cer_del
0,kresnik-korean,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,거주인 구은 줄었습니다 그지만 서울로 침택를 나는 인구 줄어들지 않았으니그래서 실제...,0.625,0.375,10,0,0,0.250,0.750,9,0,5
1,960h,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,how do you wen intrustin me ther gruked man fo...,1.875,-0.875,16,14,0,2.089,-1.089,56,61,0
2,speechT5,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><...,1.000,0.000,1,0,15,27.429,-26.429,56,1480,0


In [7]:
# 7. 평가 대상 인덱스 지정
target_indices = [430]

# 8. 평가 수행
all_results = []

for model_key, model_path in model_list.items():
    print(f"모델 평가: {model_key}")
    try:
        processor = Wav2Vec2Processor.from_pretrained(model_path)
        model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()

        for idx in target_indices:
            row = df.iloc[idx]
            wav_path = row["wav_path"]
            ref_text = row["text"].strip().lower()

            if not os.path.exists(wav_path):
                print(f"[{idx}] 파일 없음: {wav_path}")
                continue

            audio, _ = librosa.load(wav_path, sr=16000)
            inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
            with torch.no_grad():
                logits = model(**inputs).logits
            pred_ids = torch.argmax(logits, dim=-1)
            pred_text = processor.batch_decode(pred_ids)[0].strip().lower()

            wer, wer_acc, s1, i1, d1 = wer_detail(ref_text, pred_text)
            cer, cer_acc, s2, i2, d2 = cer_detail(ref_text, pred_text)

            all_results.append({
                "model": model_key,
                "index": idx,
                "ref_text": ref_text,
                "pred_text": pred_text,
                "wer": round(wer, 3), "wer_acc": round(wer_acc, 3),
                "wer_sub": s1, "wer_ins": i1, "wer_del": d1,
                "cer": round(cer, 3), "cer_acc": round(cer_acc, 3),
                "cer_sub": s2, "cer_ins": i2, "cer_del": d2
            })
    except Exception as e:
        print(f"{model_key} 평가 실패:", e)

# 9. 결과 출력
result_df = pd.DataFrame(all_results)
display(result_df)

모델 평가: kresnik-korean
모델 평가: kresnik-korean-v2
kresnik-korean-v2 평가 실패: kresnik/wav2vec2-large-xlsr-53-korean-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
모델 평가: xlsr-53


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


xlsr-53 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: xlsr-300m
xlsr-300m 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: xlsr-1b
xlsr-1b 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: xlsr-2b
xlsr-2b 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: 960h


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 평가: hubert-large-ls960-ft


You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.


hubert-large-ls960-ft 평가 실패: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?
모델 평가: hubert-base-ls960
hubert-base-ls960 평가 실패: expected str, bytes or os.PathLike object, not NoneType
모델 평가: speechT5


You are using a model of type speecht5 to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at microsoft/speecht5_asr and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.layer_norm.bias', 'wav2vec2.encoder.layer_norm.weight', 'wav2vec2.encoder.layers.0.attention.k_proj.bias', 'wav2vec2.encoder.layers.0.attention.k_proj.weight', 'wav2vec2.encoder.layers.0.attention.out_proj.bias', 'wav2vec2.encoder.layers.0.attention.out_proj.weight', 'wav2vec2.encoder.layers.0.attention.q_proj.bias', 'wav2vec2.encoder.layers.0.attention.q_proj.weight', 'wav2vec2.encoder.layers.0.attention.v_proj.bias', 'wav2vec2.encoder.layers.0.attention.v_proj.weight', 'wav2vec2.encoder.layers.0.feed_forward.intermediate_dense.bias', 'wav2vec2.encoder.layers.0.feed_forward.intermediate_dense.weight', 'wav2vec2.encoder.layers.0.feed_forward.output_de

모델 평가: kocohub-wav2vec2-ko
kocohub-wav2vec2-ko 평가 실패: kocohub/wav2vec2-large-ko-ft is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
모델 평가: etri-korean
etri-korean 평가 실패: etri/wav2vec2-large-korean is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


,model,index,ref_text,pred_text,wer,wer_acc,wer_sub,wer_ins,wer_del,cer,cer_acc,cer_sub,cer_ins,cer_del
0,kresnik-korean,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,거주인 구은 줄었습니다 그지만 서울로 침택를 나는 인구 줄어들지 않았으니그래서 실제...,0.625,0.375,10,0,0,0.250,0.750,9,0,5
1,960h,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,how do you wen intrustin me ther gruked man fo...,1.875,-0.875,16,14,0,2.089,-1.089,56,61,0
2,speechT5,430,거주 인구는 줄었습니다 그렇지만 서울로 출퇴근하는 인구는 줄어들지 않았습니다. 그래...,my —-jjjfftaogoojætmjhoildpatjj-jjbdfot-jitlgj...,1.000,0.000,13,0,3,8.982,-7.982,56,447,0


pre-trained 모델을 whisher를 사용하여 ASR모델 구현

whisper모델은 한국어를 대상으로 해도 원활하게 음성 인식해준다.

Whisper Model STT evaluation

In [8]:
# 1. 데이터 다운로드
!wget -O KoreanReadSpeechCorpus.tar.gz https://www.openslr.org/resources/97/KoreanReadSpeechCorpus.tar.gz

# 2. 압축 해제
!tar -xvzf KoreanReadSpeechCorpus.tar.gz

--2025-07-18 12:39:37--  https://www.openslr.org/resources/97/KoreanReadSpeechCorpus.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/97/KoreanReadSpeechCorpus.tar.gz [following]
--2025-07-18 12:39:38--  https://openslr.elda.org/resources/97/KoreanReadSpeechCorpus.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281297310 (268M) [application/x-gzip]
Saving to: ‘KoreanReadSpeechCorpus.tar.gz’

KoreanReadSpeechCor 100%[===================>] 268.27M  23.9MB/s    in 13s     

2025-07-18 12:39:51 (21.2 MB/s) - ‘KoreanReadSpeechCorpus.tar.gz’ saved [281297310/281297310]

AirbnbStudio/
AirbnbStudio/sub100100a00000.wav
Air

In [9]:
#전사 목록과 음원 목록 매핑
import json
import pandas as pd

# JSON 파일 로드
json_path = "/content/Korean_Read_Speech_Corpus_sample.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 모든 발화의 (파일 경로, 텍스트) 추출
entries = []
for location, utterances in data.items():
    for uid, info in utterances.items():
        wav_path = f"/content/{location}/{uid}.wav"
        text = info["text"]
        entries.append({"wav_path": wav_path, "text": text})

# DataFrame 생성
df = pd.DataFrame(entries)

# 결과 확인
print(df.head())

                                    wav_path  \
0  /content/AirbnbStudio/sub100100a00000.wav   
1  /content/AirbnbStudio/sub100100a00001.wav   
2  /content/AirbnbStudio/sub100100a00002.wav   
3  /content/AirbnbStudio/sub100100a00003.wav   
4  /content/AirbnbStudio/sub100100a00004.wav   

                                                text  
0                                저 식당 음식이 정말 맛있나 봐요.  
1                  아, 저기요. 삼계탕만 파는 식당인데 항상 사람들이 많아요.  
2   우리 회사 근처에 저런 유명한 식당이 있었네요. 다음에 삼계탕 한번 먹으러 가야겠어요.  
3  저 식당은 그날 준비한 걸 다 팔면 문을 닫아요. 그러니까 늦게 가면 못 드실 수도...  
4             여권을 만들어야 하는데요. 회사 일이 늦게 끝나서 갈 시간이 없어요.  


In [ ]:
!pip install -q transformers librosa

import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Whisper 모델 로드 (large-v2 모델)
model_name = "openai/whisper-large-v2"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# 인덱스를 지정해 Whisper로 STT 수행
def infer_whisper(index):
    if index < 0 or index >= len(df):
        print("잘못된 인덱스입니다.")
        return

    path = df.iloc[index]["wav_path"]
    print(f"[{index}] 정답 문장: {df.iloc[index]['text']}")

    # Whisper는 16kHz 모노 입력을 기대
    audio, sr = librosa.load(path, sr=16000)

    # Whisper는 float32 numpy ([-1, 1] 범위) 입력을 받음
    input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(model.device)

    # 디코딩 (자동 언어 감지 및 한국어 추론)
    predicted_ids = model.generate(input_features)
    result = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    print("Whisper 인식 결과:", result)

# 실행 예시
infer_whisper(230)


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]